# Agent Orchestration with a Planner

In this workshop, we will use the planner to design a multi-agent **FLEET** designated for the task of generating a cybersecurity report. The following agents will be included:

1. time_keeper
2. cyber_collector
3. db_reader
4. data_analyzer
5. security_evaluator
6. report_writer

In [1]:
import os
import logging
from dotenv import load_dotenv


logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(
    logging.WARNING)

load_dotenv()
print(os.getenv('CHAT_MODEL'))

gpt-4o-mini


# List all agents

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

In [3]:
fleet_name = "internet_threat_analysis"
agent_fleet = []
agent_list = project_client.agents.list_agents().data
for _agent in agent_list:
    if "group" in _agent.metadata.keys() and _agent.metadata["group"] == fleet_name:
        agent_fleet.append({"id": _agent.id, 
                            "name": _agent.name,
                            "description": _agent.description})
        
agent_fleet

[{'id': 'asst_UJjhDpUdM5rbSpNaBcWD28La',
  'name': 'report_writer',
  'description': 'An advanced AI agent responsible for integrating information and analysis results from various agents to produce a professional, detailed, and actionable cybersecurity report.'},
 {'id': 'asst_yFHWy79liAvfwsdGKQTcGqAp',
  'name': 'security_evaluator',
  'description': 'AI agent that assists users in generating radar charts for cybersecurity metrics.'},
 {'id': 'asst_TZ8XzqpP0rld787V80Nv3Yhb',
  'name': 'data_analyzer',
  'description': 'An agent that analyzes data using forecasting or anomaly detection.'},
 {'id': 'asst_OADxbrRxRnY3PgylKysvgp34',
  'name': 'db_reader',
  'description': 'This agent fetches data from a specified SQLite database table, saves it as a CSV file, and returns the status and file path in JSON format.'},
 {'id': 'asst_bn1DO0L5aTqw604kfLerrZgU',
  'name': 'cyber_collector',
  'description': 'The cyber collector can collect information about cyber security from its knowledge base

In [4]:
aoai_client = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01")

deployment_name = os.environ["CHAT_MODEL"]

### System Prompt

您是一個智能助手，能夠回應用戶的查詢。當您收到用戶的消息時，您需要判斷是否可以直接回答，或者是否更適合分配其中一位列出的代理來回應。

[__TERMINATION__] 如果您認為用戶的問題已經完全回答，請以[__TERMINATION__]作為前綴，然後回應您的回答。您的回答應提供所有相關信息和下載鏈接，以符合用戶的查詢，並附上結尾信息。不要僅僅總結對話，而是提供一個清晰簡潔的答案，包括相關鏈接。

直接回應：如果問題在您的知識和能力範圍內，您將直接回應。您的回應應以前綴 [__PLANNER__] 開頭，然後是您的答案。

代理分配：如果查詢更適合由某位代理處理，您將根據用戶的消息分配最合適的代理。您的回應將以前綴 [__AGENT__] 開頭，然後是代理的名字。如果所分配的代理無法提供滿意的答案，您可以決定分配另一位可能更好地滿足用戶需求的代理。
這是您將要合作的代理列表：

[__代理列表佔位符__]

在決定分配哪個代理時，請務必考慮每個代理的描述。如果某個代理無法找到足夠的信息或提供相關的回應，您可以分配另一個代理，以確保用戶獲得最相關和最有見地的答案。您的目標是通過您自己的知識或通過正確的代理，向用戶提供清晰和有幫助的信息。
## 限制條件
- 如果您可以直接回答問題，則不要分配代理。
- 如果有代理人能夠回答問題，請分配該代理人。
- 如果代理人無法找到足夠的信息或提供相關的回應，您可以分配另一位代理人，以確保用戶獲得最相關和最有見地的答案。
- 如果您不分配代理人，您必須以前綴 [__PLANNER__] 開頭，然後是您的回應。例如，[__PLANNER__]您好，我在這裡協助您。
- 如果您分配了代理人，您必須以前綴 [__AGENT__] 開頭，然後是代理人的名字，接著是 [__TASK__]，然後是對該代理人的請求。例如：[__AGENT__]人力資源幫助台；[__TASK__]獲取公司福利政策。

In [5]:
sys_prompt = """You are an intelligent assistant capable of responding to user queries. When you receive a message from the user, you need to determine whether you can answer it directly or if it would be more appropriate to allocate one of the listed agents to respond.

Direct Response: If the question falls within your knowledge and capabilities, you will respond directly. Your response should start with the prefix [__PLANNER__], followed by your answer.

Agent Allocation: If the query is better suited for one of the agents, you will allocate the most appropriate agent based on the user’s message. Your response will begin with the prefix [__AGENT__], followed by the agent's name. If the allocated agent is unable to provide a satisfactory answer, you may decide to allocate a different agent that may better address the user's needs.

Here is the list of agents you will be working with:
[__AGENT_LIST_PLACEHOLDER__]

Make sure to consider the descriptions of each agent when deciding which one to allocate. If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer. Your goal is to provide clear and helpful information to the user, whether through your own knowledge or via the correct agent.

## Constraints
- Do not allocate an agent if you can answer the question directly.
- If there is an agent that can possibly answer the question, allocate that agent.
- If an agent cannot find sufficient information or provide a relevant response, you may allocate another agent to ensure the user receives the most relevant and informed answer.
- If you don't allocate an agent, you must respond with the prefix [__PLANNER__], followed by your response. For example, [__PLANNER__]Hello, I'm here to assist you.;
- If you allocate an agent, you must respond with the prefix [__AGENT__], followed by the agent's name, and then [__TASK__] followed by the request to that agent. For example: [__AGENT__]HR Helpdesk;[__TASK__]Get the company benefit policies.;
- If you think the user's question is fully answered, respond with the prefix [__TERMINATION__] followed by your response. Your response should provide all relevant information and download links matching the user's query, followed by a closing message. Don't just make a summary of the conversation, but provide a clear and concise answer to the user's query including relevant links.
"""

In [6]:
import json

cyber_fleet_str = json.dumps(agent_fleet).replace("{", '{{').replace("}", '}}')
print(sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str))

You are an intelligent assistant capable of responding to user queries. When you receive a message from the user, you need to determine whether you can answer it directly or if it would be more appropriate to allocate one of the listed agents to respond.

Direct Response: If the question falls within your knowledge and capabilities, you will respond directly. Your response should start with the prefix [__PLANNER__], followed by your answer.

Agent Allocation: If the query is better suited for one of the agents, you will allocate the most appropriate agent based on the user’s message. Your response will begin with the prefix [__AGENT__], followed by the agent's name. If the allocated agent is unable to provide a satisfactory answer, you may decide to allocate a different agent that may better address the user's needs.

Here is the list of agents you will be working with:
[{{"id": "asst_UJjhDpUdM5rbSpNaBcWD28La", "name": "report_writer", "description": "An advanced AI agent responsible f

- 產生報告的準則
1. 從計時器開始獲取當前時間。
2. 分配 Cyber Collector 以檢索最新的威脅資訊。
3. 使用 cyber collector 中的指標類型讓資料庫讀取器獲取資料庫數據。
4. 使用資料庫讀取器中的 CSV 檔讓數據分析器執行分析，包括預測和異常檢測。
5. 分配安全評估員以創建評估指標的雷達圖。
6. 啟動報表編寫器以編譯資訊並生成 PDF 報表。

In [7]:
addtional_constraint = '''
## Guidelines to generate a report
1. Start with the time keeper to get the current time.
2. Allocate the cyber collector to retrieve latest threat information.
3. Use the metric types from the cyber collector to have the db reader fetch database data.
4. Use the CSV files from the db reader to have the data analyzer perform analysis, including forecasting and anomaly detection.
5. Allocate the security evaluator to create radar chart of evaluation metrics.
6. Start report writer to compile information and generate PDF report.
'''

In [8]:
from user_functions import user_functions
from azure.ai.projects.models import FunctionTool, RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

functions = FunctionTool(functions=user_functions)


def agent_execution(agent_id, task, context):

    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id, role="user", content=f'task: {task} \n\n context: {context}')

    print(f"Created message, ID: {message.id}")

    # Create and process assistant run in thread with tools
    run = project_client.agents.create_run(
        thread_id=thread.id, agent_id=agent_id)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.cancel_run(
                    thread_id=thread.id, run_id=run.id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        print(f"Executing tool call: {tool_call}")
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(
                                tool_call_id=tool_call.id,
                                output=output,
                            )
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            print(f"Tool outputs: {tool_outputs}")
            if tool_outputs:
                project_client.agents.submit_tool_outputs_to_run(
                    thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    return messages

In [9]:
def get_assistant_content(agent_messages):
    contents = []
    for message in agent_messages:
        if message['role'] == 'assistant':
            contents.append(message['content'][0]['text']['value'])
    return contents

In [10]:
messages = [
    {
        "role": "system",
        "content": sys_prompt.replace("[__AGENT_LIST_PLACEHOLDER__]", cyber_fleet_str) + addtional_constraint
    },
    {
        "role": "user",
        "content": "Generate a cybersecurity report for Contoso. The security evaluation metrics are: Vulnerability Score(8), Detection Rate(7), Response Time(6), Threat Intelligence(9), System Uptime(8)"
    },
]

In [11]:
response = aoai_client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    temperature=0.7,
    max_tokens=1000
)

llm_response = response.choices[0].message.content

messages.append({"role": "assistant", "content": f"{llm_response}"})
while "[__AGENT__]" in llm_response:
    agent_name = llm_response.split("[__AGENT__]")[1].split(";")[0]
    print(f"Allocated agent: {agent_name}")
    agent_task = llm_response.split("[__TASK__]")[1]
    print(f"Agent task: {agent_task}")
    # find agent id from agent fleet based on agent_name
    agent_id = None
    for agent in agent_fleet:
        if agent["name"] == agent_name:
            agent_id = agent["id"]
            break

    if agent_id:
        agent_messages = agent_execution(agent_id, agent_task, json.dumps(messages[1:]))
        agent_contents = get_assistant_content(agent_messages.data)
        if agent_contents:
            for agent_content in agent_contents:
                messages.append(
                    {"role": "assistant", "content": f"[__AGENT__({agent_name})]{agent_content}"})

        response = aoai_client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            temperature=0.7,
            max_tokens=1000
        )
        llm_response = response.choices[0].message.content
        print(llm_response)
        messages.append({"role": "assistant", "content": f"{llm_response}"})
    else:
        break

print(messages)

Allocated agent: time_keeper
Agent task: Get the current time.
Created thread, ID: thread_dfaPBWXqjwOOoORRryABgB8Q
Created message, ID: msg_YHWU9gdwG9xWdJ4siEcAGwkU
Created run, ID: run_QCzYehybn84ZqOwhrW7EohUX
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Executing tool call: {'id': 'call_mseCKmBJGS8Kn2F6S23Vsc5Z', 'type': 'function', 'function': {'name': 'fetch_current_datetime', 'arguments': '{}'}}
Tool outputs: [{'tool_call_id': 'call_mseCKmBJGS8Kn2F6S23Vsc5Z', 'output': '{"current_time": "2025-03-27 19:17:32"}'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED
[__AGENT__]cyber_collector;[__TASK__]Retrieve the latest threat information for cybersecurity.
Allocated agent: cyber_collector
Agent task: Retrieve the latest threat information for cybersecurity.
Created thread, ID: thread_kW9WSMzxXTO3YTLh8yaq9liK
Created messa

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Attempt 1 failed...
INFO:nixtla.nixtla_client:Attempt 2 failed...
INFO:nixtla.nixtla_client:Attempt 3 failed...
INFO:nixtla.nixtla_client:Attempt 4 failed...
INFO:nixtla.nixtla_client:Attempt 5 failed...
INFO:nixtla.nixtla_client:Attempt 6 failed...


Executing tool call: {'id': 'call_zlfUXEpvumfPcYtw6yL3WYeb', 'type': 'function', 'function': {'name': 'analyze_data', 'arguments': '{"file_path": "./data/intrusion_attempts_20250327_191750.csv", "method": "anomaly_detection"}'}}


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Anomaly Detector Endpoint...
INFO:nixtla.nixtla_client:Attempt 1 failed...


KeyboardInterrupt: 